# Face Recognition

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

## Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset_celebs/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset_celebs/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

### Transfer Learning

In [ ]:
from keras.applications import VGG19
base_model = VGG19(include_top = False, input_shape = (64,64,3), classes = 3)
for layers in base_model.layers:
    layers.trainable=False    
model= tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Dense(1024,activation='relu'))
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dense(248,activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

model.summary()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x = training_set, validation_data = test_set, epochs = 25)

### Saving Model

In [ ]:
model.save('transfer_learning')

### Load Model

In [ ]:
model = tf.keras.models.load_model('transfer_learning')

## Building own CNN

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

### Saving Model

In [ ]:
cnn.save('cnn')

### Loading Model

In [ ]:
model = tf.keras.models.load_model('cnn')

## Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset_celebs/single_prediction/e.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
#print(result)

predicted_class_indices=np.argmax(result,axis=1)
#print(predicted_class_indices)

labels = (training_set.class_indices)
#print(labels.items())

labels = dict((v,k) for k,v in labels.items())
#print(labels)

predictions = [labels[k] for k in predicted_class_indices]
print(predictions)

In [ ]:
from PIL import Image
import cv2
import numpy as np
from keras.preprocessing import image

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    if faces is ():
        cv2.putText(frame,'No face found', (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,0), 2)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        face = frame[y:y+h, x:x+w]

        if type(face) is np.ndarray:
            face = cv2.resize(face, (64, 64))
            face = image.img_to_array(face)
            face = np.expand_dims(face, axis = 0)
            result = model.predict(face)
            predicted_class_indices=np.argmax(result,axis=1)
            labels = (training_set.class_indices)
            labels = dict((v,k) for k,v in labels.items())
            predictions = [labels[k] for k in predicted_class_indices]
            print(predictions)
            name=predictions[0]
        
            if(name):
                cv2.rectangle(frame,(x,y-12),(x+w,y+12),(0,255,255),-1)
                cv2.putText(frame,name, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()